# Geo anlaysis of Toronto
### Capstone project, Week 3

## Task 1 import the neighborhood info from wikipedia page and save it in a dataframe

First of all, let's import the packages.

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd

Use requests to get the content from the url

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)
response #if the connection is good then the result should be <Response [200]>

<Response [200]>

Use beautiful soup to parse the page. We only need the first table that contains neighborhood information.

In [3]:
soup = BeautifulSoup(response.text, "html.parser")
table = soup.findAll('table')[0] #only parse the first table

convert the tabel into a dataframe and add the headings

In [4]:
df = []
for rows in table.findAll('tr'):
    row = []
    for items in rows.findAll('td'):
        row.append(items.text.replace('\n',''))
    df.append(row)
df.pop(0)
df = pd.DataFrame(df, columns=['Postal Code', 'Borough', 'Neighborhood'])
print(df.head(),"\n\n", df.shape)

  Postal Code           Borough      Neighborhood
0         M1A      Not assigned      Not assigned
1         M2A      Not assigned      Not assigned
2         M3A        North York         Parkwoods
3         M4A        North York  Victoria Village
4         M5A  Downtown Toronto      Harbourfront 

 (288, 3)


Remove the rows with Borough being "Not assigned"

In [5]:
df = df[df['Borough'] != 'Not assigned']


If Borough exists but Neighborhood is not assigned, make the neighborhood the same as borough.

In [6]:
df.loc[df['Neighborhood'] == 'Not assigned','Neighborhood'] = df.loc[df['Neighborhood'] == 'Not assigned','Borough']

More than one neighborhood can exist in one postal code area. These rows will be combined into one row with the neighborhoods separated with a comma.

In [7]:
df = df.groupby(['Postal Code','Borough'])['Neighborhood'].agg(lambda col: ','.join(col)).reset_index()

Show the shape of cleaned dataframe

In [8]:
df.shape

(103, 3)

## Task 2 get the geo data and merge the dataframes

I tried the geocode package, but it failed. So I just use the csv file provided.

In [9]:
url = "https://cocl.us/Geospatial_data"
geodf = pd.read_csv(url)

In [10]:
df = pd.merge(df, geodf, on='Postal Code')

In [11]:
df.head()

Postal Code      Borough                          Neighborhood   Latitude  \
0         M1B  Scarborough                         Rouge,Malvern  43.806686   
1         M1C  Scarborough  Highland Creek,Rouge Hill,Port Union  43.784535   
2         M1E  Scarborough       Guildwood,Morningside,West Hill  43.763573   
3         M1G  Scarborough                                Woburn  43.770992   
4         M1H  Scarborough                             Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

## Task 3 Perform some analysis like we did to New York

Import the packages first.

In [12]:
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import numpy as np

Here we only perform analysis with the borough with 'Toronto' in the names.

In [13]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
location

Location(Toronto, Golden Horseshoe, Ontario, M6K 1X9, Canada, (43.653963, -79.387207, 0.0))

In [24]:
dt_data = df[(df['Borough'] == 'Downtown Toronto') | (df['Borough'] == 'Central Toronto') | \
            (df['Borough'] == 'West Toronto') | (df['Borough'] == 'East Toronto')].reset_index(drop=True)
print(dt_data.head(),dt_data.shape)

  Postal Code          Borough                   Neighborhood   Latitude  \
0         M4E     East Toronto                    The Beaches  43.676357   
1         M4K     East Toronto    The Danforth West,Riverdale  43.679557   
2         M4L     East Toronto  The Beaches West,India Bazaar  43.668999   
3         M4M     East Toronto                Studio District  43.659526   
4         M4N  Central Toronto                  Lawrence Park  43.728020   

   Longitude  
0 -79.293031  
1 -79.352188  
2 -79.315572  
3 -79.340923  
4 -79.388790   (38, 5)


In [28]:
# create map of Old Toronto using latitude and longitude values
map_toronto = folium.Map(location=[location.latitude, location.longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(dt_data['Latitude'], dt_data['Longitude'], \
                                           dt_data['Borough'], dt_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Foursquare credentials

Foursquare credentials

In [33]:
CLIENT_ID = 'NELDHJBYBEKTRSPQ3FSZIQAX2FIQMPTUPTJC3G51J2IUYPBP' # your Foursquare ID
CLIENT_SECRET = 'PZUK3RM22COSNA3JXB0KVXFLR1TW0PVQRCPFVPHGMLRO4YJN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [34]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    dt_data['Latitude'][0], 
    dt_data['Longitude'][0], 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=NELDHJBYBEKTRSPQ3FSZIQAX2FIQMPTUPTJC3G51J2IUYPBP&client_secret=PZUK3RM22COSNA3JXB0KVXFLR1TW0PVQRCPFVPHGMLRO4YJN&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [35]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d3612dddb1d810039ae3ee3'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bd461bc77b29c74a07d9282-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/hikingtrail_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d159941735',
         'name': 'Trail',
         'pluralName': 'Trails',
         'primary': True,
         'shortName': 'Trail'}],
       'id': '4bd461bc77b29c74a07d9282',
       'location': {'address': 'Glen Manor',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Queen St.',
        'distance': 89,
        'formattedAddress': ['Glen Manor (Queen St.)', 'Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682

Get the category name of the nearest venues

In [36]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [37]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name            categories        lat  \
0                   Glen Manor Ravine                 Trail  43.676821   
1  The Big Carrot Natural Food Market     Health Food Store  43.678879   
2                 Grover Pub and Grub                   Pub  43.679181   
3                 Glen Stewart Ravine  Other Great Outdoors  43.676300   
4                       Upper Beaches          Neighborhood  43.680563   

         lng  
0 -79.293942  
1 -79.297734  
2 -79.297215  
3 -79.294784  
4 -79.292869

In [38]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


Get venues for all the neighborhood.

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [40]:
dt_toronto_venues = getNearbyVenues(names=dt_data['Neighborhood'],
                                   latitudes=dt_data['Latitude'],
                                   longitudes=dt_data['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [41]:
print(dt_toronto_venues.shape)
dt_toronto_venues.head()

(1707, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  The Beaches              43.676357              -79.293031   
1  The Beaches              43.676357              -79.293031   
2  The Beaches              43.676357              -79.293031   
3  The Beaches              43.676357              -79.293031   
4  The Beaches              43.676357              -79.293031   

                                Venue  Venue Latitude  Venue Longitude  \
0                   Glen Manor Ravine       43.676821       -79.293942   
1  The Big Carrot Natural Food Market       43.678879       -79.297734   
2                 Grover Pub and Grub       43.679181       -79.297215   
3                 Glen Stewart Ravine       43.676300       -79.294784   
4                       Upper Beaches       43.680563       -79.292869   

         Venue Category  
0                 Trail  
1     Health Food Store  
2                   Pub  
3  Other Great Outdoors  
4          Neighborhood

In [42]:
dt_toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Adelaide,King,Richmond                                                100   
Berczy Park                                                            57   
Brockton,Exhibition Place,Parkdale Village                             25   
Business Reply Mail Processing Centre 969 Eastern                      18   
CN Tower,Bathurst Quay,Island airport,Harbourfr...                     16   
Cabbagetown,St. James Town                                             44   
Central Bay Street                                                     84   
Chinatown,Grange Park,Kensington Market                               100   
Christie                                                               15   
Church and Wellesley                                                   89   
Commerce Court,Victoria Hotel                                         100   
Davisville                                                             35   
Davisville North                                                        7   
Deer Park,Forest Hill SE,Rathnelly,South Hill,S...                     15   
Design Exchange,Toronto Dominion Centre                               100   
Dovercourt Village,Dufferin                                            18   
First Canadian Place,Underground city                                 100   
Forest Hill North,Forest Hill West                                      4   
Harbord,University of Toronto                                          34   
Harbourfront East,Toronto Islands,Union Station                       100   
Harbourfront,Regent Park                                               51   
High Park,The Junction South                                           22   
Lawrence Park                                                           3   
Little Portugal,Trinity                                                64   
Moore Park,Summerhill East                                              1   
North Toronto West                                                     19   
Parkdale,Roncesvalles                                                  15   
Rosedale                                                                5   
Roselawn                                                                1   
Runnymede,Swansea                                                      37   
Ryerson,Garden District                                               100   
St. James Town                                                        100   
Stn A PO Boxes 25 The Esplanade                                        95   
Studio District                                                        42   
The Annex,North Midtown,Yorkville                                      24   
The Beaches                                                             5   
The Beaches West,India Bazaar                                          21   
The Danforth West,Riverdale                                            41   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Adelaide,King,Richmond                                                 100   
Berczy Park                                                             57   
Brockton,Exhibition Place,Parkdale Village                              25   
Business Reply Mail Processing Centre 969 Eastern                       18   
CN Tower,Bathurst Quay,Island airport,Harbourfr...                      16   
Cabbagetown,St. James Town                                              44   
Central Bay Street                                                      84   
Chinatown,Grange Park,Kensington Market                                100   
Christie                                                                15   
Church and Wellesley                                                    89   
Commerce Court,Victoria Hotel      

In [43]:
print('There are {} uniques categories.'.format(len(dt_toronto_venues['Venue Category'].unique())))

There are 232 uniques categories.


Encode the categories.

In [44]:
# one hot encoding
dt_toronto_onehot = pd.get_dummies(dt_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

dt_toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True)

In [45]:
dt_toronto_onehot = pd.concat([dt_toronto_venues['Neighborhood'],dt_toronto_onehot], axis = 1)
dt_toronto_onehot.head()

Neighborhood  Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0  The Beaches                  0        0                   0             0   
1  The Beaches                  0        0                   0             0   
2  The Beaches                  0        0                   0             0   
3  The Beaches                  0        0                   0             0   
4  The Beaches                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop     ...       Thrift / Vintage Store  Toy / Game Store  Trail  \
0             0     ...                            0                 0      1   
1             0     ...                            0                 0      0   
2             0     ...                            0                 0      0   
3             0     ...                            0                 0      0   
4             0     ...                            0                 0      0   

   Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0              0                              0                 0   
1              0                              0                 0   
2              0                              0                 0   
3              0                              0                 0   
4              0                              0                 0   

   Vietnamese Restaurant  Wine Bar  Wings Joint  Yoga Studio  
0                      0         0            0            0  
1                      0         0            0            0  
2                      0         0            0            0  
3                      0         0            0            0  
4                      0         0            0            0  

[5 rows x 232 columns]

In [46]:
dt_toronto_onehot.shape

(1707, 232)

In [47]:
dt_toronto_grouped = dt_toronto_onehot.groupby('Neighborhood').mean().reset_index()
dt_toronto_grouped

Neighborhood  Afghan Restaurant  \
0                              Adelaide,King,Richmond           0.000000   
1                                         Berczy Park           0.000000   
2          Brockton,Exhibition Place,Parkdale Village           0.000000   
3   Business Reply Mail Processing Centre 969 Eastern           0.000000   
4   CN Tower,Bathurst Quay,Island airport,Harbourf...           0.000000   
5                          Cabbagetown,St. James Town           0.000000   
6                                  Central Bay Street           0.000000   
7             Chinatown,Grange Park,Kensington Market           0.000000   
8                                            Christie           0.000000   
9                                Church and Wellesley           0.011236   
10                      Commerce Court,Victoria Hotel           0.000000   
11                                         Davisville           0.000000   
12                                   Davisville North           0.000000   
13  Deer Park,Forest Hill SE,Rathnelly,South Hill,...           0.000000   
14            Design Exchange,Toronto Dominion Centre           0.000000   
15                        Dovercourt Village,Dufferin           0.000000   
16              First Canadian Place,Underground city           0.000000   
17                 Forest Hill North,Forest Hill West           0.000000   
18                      Harbord,University of Toronto           0.000000   
19    Harbourfront East,Toronto Islands,Union Station           0.000000   
20                           Harbourfront,Regent Park           0.000000   
21                       High Park,The Junction South           0.000000   
22                                      Lawrence Park           0.000000   
23                            Little Portugal,Trinity           0.000000   
24                         Moore Park,Summerhill East           0.000000   
25                                 North Toronto West           0.000000   
26                              Parkdale,Roncesvalles           0.000000   
27                                           Rosedale           0.000000   
28                                           Roselawn           0.000000   
29                                  Runnymede,Swansea           0.000000   
30                            Ryerson,Garden District           0.000000   
31                                     St. James Town           0.000000   
32                    Stn A PO Boxes 25 The Esplanade           0.000000   
33                                    Studio District           0.000000   
34                  The Annex,North Midtown,Yorkville           0.000000   
35                                        The Beaches           0.000000   
36                      The Beaches West,India Bazaar           0.000000   
37                        The Danforth West,Riverdale           0.000000   

    Airport  Airport Food Court  Airport Gate  Airport Lounge  \
0    0.0000              0.0000        0.0000           0.000   
1    0.0000              0.0000        0.0000           0.000   
2    0.0000              0.0000        0.0000           0.000   
3    0.0000              0.0000        0.0000           0.000   
4    0.0625              0.0625        0.0625           0.125   
5    0.0000              0.0000        0.0000           0.000   
6    0.0000              0.0000        0.0000           0.000   
7    0.0000              0.0000        0.0000           0.000   
8    0.0000              0.0000        0.0000           0.000   
9    0.0000              0.0000        0.0000           0.000   
10   0.0000              0.0000        0.0000           0.000   
11   0.0000              0.0000        0.0000           0.000   
12   0.0000              0.0000        0.0000           0.000   
13   0.0000              0.0000        0.0000           0.000   
14   0.0000              0.0000        0.0000           0.000   
15   0.0000              0.0000     

Find out the most frequent venue categories in each neighborhood.

In [48]:
num_top_venues = 5

for hood in dt_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dt_toronto_grouped[dt_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2  Thai Restaurant  0.04
3       Steakhouse  0.04
4              Bar  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.11
1        Cocktail Bar  0.05
2          Steakhouse  0.04
3  Seafood Restaurant  0.04
4      Farmers Market  0.04


----Brockton,Exhibition Place,Parkdale Village----
                   venue  freq
0                   Café  0.08
1  Performing Arts Venue  0.08
2            Coffee Shop  0.08
3         Breakfast Spot  0.08
4          Burrito Place  0.04


----Business Reply Mail Processing Centre 969 Eastern----
              venue  freq
0       Yoga Studio  0.06
1  Recording Studio  0.06
2        Smoke Shop  0.06
3           Brewery  0.06
4               Spa  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.19
1    Airpor

In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [50]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dt_toronto_grouped['Neighborhood']

for ind in np.arange(dt_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                             Adelaide,King,Richmond           Coffee Shop   
1                                        Berczy Park           Coffee Shop   
2         Brockton,Exhibition Place,Parkdale Village        Breakfast Spot   
3  Business Reply Mail Processing Centre 969 Eastern           Yoga Studio   
4  CN Tower,Bathurst Quay,Island airport,Harbourf...       Airport Service   

  2nd Most Common Venue  3rd Most Common Venue 4th Most Common Venue  \
0                  Café             Steakhouse                   Bar   
1          Cocktail Bar            Cheese Shop            Steakhouse   
2                  Café  Performing Arts Venue           Coffee Shop   
3                   Spa          Garden Center                Garden   
4      Airport Terminal         Airport Lounge              Boutique   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0       Thai Restaurant                   Gym          Burger Joint   
1    Seafood Restaurant              Beer Bar                  Café   
2         Grocery Store          Intersection    Falafel Restaurant   
3    Light Rail Station  Fast Food Restaurant        Farmers Market   
4           Coffee Shop         Boat or Ferry       Harbor / Marina   

  8th Most Common Venue 9th Most Common Venue   10th Most Common Venue  
0   American Restaurant      Asian Restaurant               Restaurant  
1        Farmers Market                Bakery  Comfort Food Restaurant  
2        Sandwich Place  Caribbean Restaurant               Restaurant  
3            Comic Shop                  Park              Pizza Place  
4      Sculpture Garden          Airport Gate       Airport Food Court

KMeans clustering

In [51]:
from sklearn.cluster import KMeans

In [52]:
# set number of clusters
kclusters = 5

dt_toronto_grouped_clustering = dt_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [53]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dt_toronto_merged = dt_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dt_toronto_merged = dt_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dt_toronto_merged.head() # check the last columns!

Postal Code          Borough                   Neighborhood   Latitude  \
0         M4E     East Toronto                    The Beaches  43.676357   
1         M4K     East Toronto    The Danforth West,Riverdale  43.679557   
2         M4L     East Toronto  The Beaches West,India Bazaar  43.668999   
3         M4M     East Toronto                Studio District  43.659526   
4         M4N  Central Toronto                  Lawrence Park  43.728020   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.293031               0     Health Food Store  Other Great Outdoors   
1 -79.352188               0      Greek Restaurant           Coffee Shop   
2 -79.315572               0                  Park        Sandwich Place   
3 -79.340923               0                  Café           Coffee Shop   
4 -79.388790               4                  Park              Bus Line   

  3rd Most Common Venue   4th Most Common Venue 5th Most Common Venue  \
0                 Trail                     Pub        Discount Store   
1    Italian Restaurant  Furniture / Home Store        Ice Cream Shop   
2           Pizza Place               Pet Store               Brewery   
3    Italian Restaurant                  Bakery   American Restaurant   
4           Swim School             Yoga Studio              Dive Bar   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0        Farmers Market    Falafel Restaurant           Event Space   
1           Pizza Place             Bookstore               Brewery   
2          Burger Joint         Burrito Place                   Pub   
3             Gastropub           Cheese Shop           Fish Market   
4        Farmers Market    Falafel Restaurant           Event Space   

  9th Most Common Venue 10th Most Common Venue  
0  Ethiopian Restaurant      Electronics Store  
1       Bubble Tea Shop             Restaurant  
2         Movie Theater       Sushi Restaurant  
3             Bookstore                Brewery  
4  Ethiopian Restaurant      Electronics Store

In [55]:
# create map
map_clusters = folium.Map(location=[location.latitude, location.longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_toronto_merged['Latitude'], dt_toronto_merged['Longitude'], \
                                  dt_toronto_merged['Neighborhood'], dt_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

From the map we can see 4 of 5 clusters only have 1 record, which mean we catched the outliers and make them seperate clusters. This is not a good practice. We should remove the outliers or try other algorithms.